In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/SemEval2023/SemEval2022-Task10/private_space

/content/gdrive/.shortcut-targets-by-id/1lC-ZKLaCDQyfLcof2Ak7FDa6IvTt318A/SemEval2023/SemEval2022-Task10/private_space


In [3]:
import pandas as pd
# unsupervised_data_gab = pd.read_csv("Data/starting_ki/gab_1M_unlabelled.csv")
# unsupervised_data_reddit = pd.read_csv("Data/starting_ki/reddit_1M_unlabelled.csv")

In [4]:
from sklearn.model_selection import train_test_split

target_column = "label_category"
data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")
data = data[data[target_column]!="none"]

train, test = train_test_split(data, test_size=0.2, random_state=42)
# data = train

In [5]:
if target_column == "label_category":
  label_values = [
      '1. threats, plans to harm and incitement',
      '2. derogation',
      '3. animosity',
      '4. prejudiced discussions',
  ]

#   label_map = {
#       0: '1. threats, plans to harm and incitement',
#       1: '2. derogation', 
#       2: '3. animosity', 
#       3: '4. prejudiced discussions',
#       '1. threats, plans to harm and incitement':0,
#       '2. derogation':1,
#       '3. animosity':2,
#       '4. prejudiced discussions':3,
#   }
elif target_column == "label_vector":
  label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

#   label_map = {
#       0: '1.1 threats of harm',
#       1: '1.2 incitement and encouragement of harm',
#       2: '2.1 descriptive attacks',
#       3: '2.2 aggressive and emotive attacks',
#       4: '2.3 dehumanising attacks & overt sexual objectification',
#       5: '3.1 casual use of gendered slurs, profanities, and insults',
#       6: '3.2 immutable gender differences and gender stereotypes',
#       7: '3.3 backhanded gendered compliments',
#       8: '3.4 condescending explanations or unwelcome advice',
#       9: '4.1 supporting mistreatment of individual women',
#       10: '4.2 supporting systemic discrimination against women as a group',
#       '1.1 threats of harm': 0,
#       '1.2 incitement and encouragement of harm': 1,
#       '2.1 descriptive attacks': 2,
#       '2.2 aggressive and emotive attacks': 3,
#       '2.3 dehumanising attacks & overt sexual objectification': 4,
#       '3.1 casual use of gendered slurs, profanities, and insults': 5,
#       '3.2 immutable gender differences and gender stereotypes': 6,
#       '3.3 backhanded gendered compliments': 7,
#       '3.4 condescending explanations or unwelcome advice': 8,
#       '4.1 supporting mistreatment of individual women': 9,
#       '4.2 supporting systemic discrimination against women as a group': 10,
#   }
# else:
#   raise Exception("Unknown label column")

## Shapley

In [6]:
!pip install -q transformers shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.0 MB/s eta 0:00:00


In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig

out_dir = f'../Models/1a_fine-tuned-bert'
model = BertForSequenceClassification.from_pretrained(out_dir)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [9]:
import transformers
import torch
import numpy as np
import scipy as sp

In [10]:
import shap
# define a prediction function
def f(texts):
  text_ids = [tokenizer.encode(text, max_length=100, padding='max_length', truncation=True) for text in texts]

  att_masks = []
  for ids in text_ids:
      masks = [int(id > 0) for id in ids]
      att_masks.append(masks)

  text_ids = torch.tensor(text_ids).to(device)
  att_masks = torch.tensor(att_masks).to(device)

  outputs = model(text_ids, attention_mask=att_masks)
  outputs = outputs[0].detach().cpu().numpy()
  scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
  
  val = sp.special.logit(scores[:,1]) # use one vs rest logit units
  return val

In [11]:
for label in label_values:
  print(label)

1. threats, plans to harm and incitement
2. derogation
3. animosity
4. prejudiced discussions


## Get Predictions from unannotated corpus

In [ ]:
# texts = unsupervised_data_gab["text"].values

In [ ]:
# texts = texts[0:10000]

In [ ]:
# from tqdm import tqdm
# text_ids = []
# for text in tqdm(texts, total=len(texts)):
#   ids = tokenizer.encode(text, max_length=100, padding='max_length', truncation=True)
#   text_ids.append(ids)

# text_ids_lengths = [len(text_ids[i]) for i in range(len(text_ids))]
# print(min(text_ids_lengths))
# print(max(text_ids_lengths))

# att_masks = []
# for ids in text_ids:
#     masks = [int(id > 0) for id in ids]
#     att_masks.append(masks)

In [ ]:
# text_ids = torch.tensor(text_ids);
# att_masks = torch.tensor(att_masks);

In [ ]:
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# batch_size = 32

# test_data = TensorDataset(text_ids, att_masks)
# test_sampler = SequentialSampler(test_data)
# test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# outputs = []
# with torch.no_grad():
#     model.eval()
#     for k, (mb_x, mb_m) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
#         mb_x = mb_x.to(device)
#         mb_m = mb_m.to(device)
#         output = model(mb_x, attention_mask=mb_m)
#         outputs.append(output[0].to('cpu'))

# outputs = torch.cat(outputs)

# _, predicted_values = torch.max(outputs, 1)
# predicted_values = predicted_values.numpy()


In [ ]:
# predicted_labels = ["sexist" if p==1 else "not sexist" for p in predicted_values]

In [ ]:
# unsupervised_data_gab["pred"] = predicted_labels

In [ ]:
# unsupervised_data_gab.to_csv("Results/predicted_data_gab.csv", index=False)

In [21]:
unsupervised_data_gab = pd.read_csv("Results/predicted_data_gab.csv")

In [22]:
unsupervised_data_gab = unsupervised_data_gab[unsupervised_data_gab["pred"]=="sexist"]

In [23]:
import pickle

def save_shap_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)

def save_lexicons(filepath, obj):
  with open(filepath, 'w', encoding="utf-8") as fin:
    for w in obj:
      fin.write(str(w)+"\n")

def get_lexicons(shap_values):
  feature_names = shap_values.abs.mean(0).feature_names
  shapley_values = shap_values.abs.mean(0).values

  candidates = [x for v, x in sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0]) if v > 0]
  return candidates

In [24]:
len(unsupervised_data_gab)

19633

In [ ]:
d = unsupervised_data_gab
d = {"text": d["text"].values}

In [ ]:
explainer = shap.Explainer(f, tokenizer)
shap_values = explainer(d, fixed_context=1, batch_size=512)
save_shap_values(f"./Results/shapley_values_gab.pickle", shap_values)
lexicons = get_lexicons(shap_values)
save_lexicons(f"./Results/shapley_lexicon_gab.txt", lexicons)
print("")

Partition explainer: 19634it [2:16:26,  2.40it/s]


In [12]:
# 250, 01:50, 110
# 500, 03:34, 214
# 1000, 07:08, 428
# 19633, 2:20:05, 8403


In [13]:
import pickle

def load_shap_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

def get_lexicons(shap_values):
  feature_names = shap_values.abs.mean(0).feature_names
  shapley_values = shap_values.abs.mean(0).values

  sorted_values = sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0])
  features = [x for v, x in sorted_values]
  values = {x:v for v, x in sorted_values}
  return features, values
  

shap_values = load_shap_values(f"./Results/shapley_values_gab.pickle")
k, v = get_lexicons(shap_values)
lexicons = k
lexicons_score = v

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Handle subwords

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
feature_names = shap_values.abs.mean(0).feature_names

cc = 0
for f in feature_names:
  if "#" in f:
    cc += 1
cc

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


1

In [26]:
from collections import defaultdict
from tqdm import tqdm

def map_subwords(words, subwords):
  sidx = 0
  widx = 0
  w = ""
  mapping = []
  while widx < len(words):
    w = w + words[widx]
    if sidx >= len(subwords):
      print(widx, w, sidx)
      print(words)
      print(subwords)

    idx = [sidx]
    s = subwords[sidx].replace("#", "")

    while len(w) < len(s):
      widx += 1
      w = w + words[widx].strip()

    # if words[0] == "layin":
    #     print(w, s)

    while s!=w and len(s) < len(w) and sidx+1 < len(subwords):
      sidx += 1
      s += subwords[sidx].replace("#", "").strip()
      idx.append(sidx)

      # if words[0] == "layin":
      #   print(f"[{w}] [{s}]", len(w), len(s), w==s)

    if len(s) > len(w):
      widx += 1
      sidx -= len(idx) - 1
      continue
      
    sidx += 1
    # print(w, s, idx)
    widx += 1
    w = ""

    mapping.append(idx)

  return mapping

In [39]:
def clean_and_tokenize(sent):
  sent = sent.lower().replace("#", "").replace("''", '"')

  sent = sent.encode("ascii", "ignore")
  sent = sent.decode()
  # sent = sent.replace("``", '"')
  # sent = sent.replace("`", "'")
  
  words = nltk.word_tokenize(sent)
  for widx, w in enumerate(words):
    if w=="``":
      words[widx] = '"'
      continue
    elif w=="''":
      words[widx] = '"'
      continue
  
  for w in words:
    vocabs.add(w)
    
  subwords = tokenizer.tokenize(sent)

  for sidx, s in enumerate(subwords):
    if s=="``":
      subwords[sidx] = '"'
      continue
    elif s=="''":
      subwords[sidx] = '"'
      continue
  return words, subwords


def merge_score(new_lexicons, idxs, subwords):
  for idx in idxs:
    w = ""
    s = 0
    for i in idx:
      sw = subwords[i].replace("#", "")
      w += sw
      if sw not in lexicons_score:
        # print(">>", sw)
        continue

      s += lexicons_score[sw]
      
    new_lexicons[w] = s

In [40]:
new_lexicons = defaultdict(int)
vocabs = set()

d = unsupervised_data_gab
texts = d["text"].values

for sent in tqdm(texts, total=len(texts)):
  try:
    words, subwords = clean_and_tokenize(sent)
    idxs = map_subwords(words, subwords)
    merge_score(new_lexicons, idxs, subwords)
  except Exception as e:
    print("Error", str(e))

  
  
  

sorted_list = sorted(new_lexicons.items(), key=lambda item: -item[1])
new_lexicons_scores = { k:v for k, v in sorted_list if v > 0}
new_lexicons_keys = [k for k, v in sorted_list if v > 0]

100%|██████████| 19633/19633 [00:19<00:00, 1012.93it/s]


In [46]:
# new_lexicons_scores

## Determine their class

In [41]:
with open("Results/PMI.pickle", 'rb') as fin:
    PMI = pickle.load(fin)

In [43]:
import json

with open('Results/lexicon_shapley.json') as fin:
    lexicon_shapley = json.load(fin)

In [47]:
vocabs = set()
for label in lexicon_shapley:
  vocabs.update(lexicon_shapley[label].keys())

In [65]:
# Estimate Threshold
import random
import numpy as np
import math

def get_PMI(PMI, wa, wb):
  PMI_counter, PMI_vocabs, PMI_Nx, PMI_Nxy = PMI

  Px = PMI_counter[(None, wa)]/PMI_Nx
  Py = PMI_counter[(None, wb)]/PMI_Nx

  if Px==0 or Py==0:
    return 0
    
  Pxy = PMI_counter[(wa, wb)]/PMI_Nxy

  delta = 1e-10
  p = max(math.log2((Pxy+delta)/(Px*Py)), 0)
  return p

# N = int(len(PMI[1])*0.5)
N = 100000
wordA = random.sample(PMI[1], N)
wordB = random.sample(PMI[1], N)
values = [get_PMI(PMI, wa, wb) for wa, wb in zip(wordA, wordB)]

In [66]:
threshold = np.quantile(values, 0.99)
threshold

13.35977390501769

In [68]:
def augment(PMI, word, candidates):
  PMI_counter, PMI_vocabs, PMI_Nx, PMI_Nxy = PMI
  if word not in candidates:
    return []

  selected = []
  for k in candidates:
    pmi = get_PMI(PMI, word, k)
    if pmi > threshold:
      selected.append((k, candidates[k]))
  return selected

augmented_lexicons = {}
for v in tqdm(vocabs, total=len(vocabs)):
  selected = augment(PMI, v, new_lexicons_scores)
  augmented_lexicons[v] = selected

100%|██████████| 1191/1191 [01:04<00:00, 18.38it/s]


In [70]:
new_lexicon_shapley = {}
for label in lexicon_shapley:
  new_lexicon_shapley[label] = {}

  lexicons = lexicon_shapley[label]

  new_lexicon = {}
  for word in lexicons:
    for new_word, shap in augmented_lexicons[word]:
      val = shap #math.sqrt(lexicons[word]*shap)
      if new_word in new_lexicon:
        new_lexicon[new_word] = min(val, new_lexicon[new_word])
      else:
        new_lexicon[new_word] = val

  for word in lexicons:
    new_lexicon_shapley[label][word] = lexicons[word]

  for word in new_lexicon:
    new_lexicon_shapley[label][word] = new_lexicon[word]

with open('Results/lexicon_shapley_augmented_v2_shap.json', 'w') as outfile:
    json.dump(new_lexicon_shapley, outfile)